# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [38]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [39]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [40]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2710010157642,
 'week52high': 189.82,
 'week52low': 122.13,
 'week52highSplitAdjustOnly': 188.38,
 'week52lowSplitAdjustOnly': 122.2,
 'week52change': 0.2886170677552607,
 'sharesOutstanding': 16882733538,
 'float': 0,
 'avg10Volume': 106243109,
 'avg30Volume': 97235170,
 'day200MovingAvg': 164.09,
 'day50MovingAvg': 168.97,
 'employees': 147728,
 'ttmEPS': 6.06,
 'ttmDividendRate': 0.9024738886425778,
 'dividendYield': 0.00558187768500694,
 'nextDividendDate': '',
 'exDividendDate': '2022-01-21',
 'nextEarningsDate': '2022-04-21',
 'peRatio': 27.13203714689815,
 'beta': 1.305627927937338,
 'maxChangePercent': 60.638668102881,
 'year5ChangePercent': 3.81428693069068,
 'year2ChangePercent': 1.678895114127812,
 'year1ChangePercent': 0.2880431544134517,
 'ytdChangePercent': -0.10428342485480854,
 'month6ChangePercent': 0.07778861061901993,
 'month3ChangePercent': -0.0749790858854929,
 'month1ChangePercent': -0.07591083830749899,
 'day30ChangePerc

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [41]:
data['year1ChangePercent']

0.2880431544134517

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [42]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#    print(symbol_strings[i])
for symbol_string in symbol_strings:
    print(symbol_string)
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [43]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings[:2]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string},&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['price'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
        ],
        index = my_columns),
            ignore_index = True
        )
        
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,140.16,0.10384,N/A
1,AAL,16.63,-0.334883,N/A
2,AAP,217.50,0.181487,N/A
3,AAPL,162.00,0.289956,N/A
4,ABBV,159.19,0.487566,N/A
...,...,...,...,...
195,FTNT,317.38,0.597843,N/A
196,FTV,64.00,-0.108142,N/A
197,GD,234.35,0.339995,N/A
198,GE,96.66,-0.099729,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [44]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,148,DVN,58.550,1.399606,N/A
1,89,CF,96.260,0.880039,N/A
2,42,APA,40.120,0.79968,N/A
3,109,COP,102.510,0.760813,N/A
4,37,ANET,132.000,0.757139,N/A
5,160,EOG,121.300,0.701767,N/A
6,110,COST,571.781,0.681811,N/A
7,195,FTNT,317.380,0.597843,N/A
8,175,FANG,140.280,0.586747,N/A
9,123,CVX,163.670,0.550042,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [45]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')
    
    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number! \nPlease try again:')
        portfolio_size = input('Enter the size of your portfolio:')
              
portfolio_input()
print(portfolio_size)

Enter the size of your portfolio:1000000
1000000


In [46]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])
    
final_dataframe

C:\Users\conno\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,148,DVN,58.550,1.399606,341
1,89,CF,96.260,0.880039,207
2,42,APA,40.120,0.79968,498
3,109,COP,102.510,0.760813,195
4,37,ANET,132.000,0.757139,151
5,160,EOG,121.300,0.701767,164
6,110,COST,571.781,0.681811,34
7,195,FTNT,317.380,0.597843,63
8,175,FANG,140.280,0.586747,142
9,123,CVX,163.670,0.550042,122


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [47]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string},&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
        pd.Series(
        [ 
            symbol,
            data[symbol]['price'],
            'N/A',
            data[symbol]['stats']['year1ChangePercent'],
            'N/A',
            data[symbol]['stats']['month6ChangePercent'],
            'N/A',
            data[symbol]['stats']['month3ChangePercent'],
            'N/A',
            data[symbol]['stats']['month1ChangePercent'],
            'N/A',
            'N/A'
            
        ],
        index = hqm_columns),
        ignore_index = True
        )
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,137.38,N/A,0.106253,N/A,-0.219643,N/A,-0.104639,N/A,-0.001130,N/A,N/A
1,AAL,17.32,N/A,-0.337113,N/A,-0.174364,N/A,-0.003154,N/A,-0.124997,N/A,N/A
2,AAP,215.96,N/A,0.184582,N/A,0.003529,N/A,-0.136586,N/A,-0.067465,N/A,N/A
3,AAPL,161.00,N/A,0.280215,N/A,0.076708,N/A,-0.074109,N/A,-0.077142,N/A,N/A
4,ABBV,165.78,N/A,0.476359,N/A,0.498195,N/A,0.199295,N/A,0.069885,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
95,CINF,132.47,N/A,0.257220,N/A,0.117668,N/A,0.117335,N/A,0.019044,N/A,N/A
96,CL,77.07,N/A,0.013296,N/A,-0.006220,N/A,-0.101421,N/A,-0.041150,N/A,N/A
97,CLX,135.46,N/A,-0.287987,N/A,-0.206446,N/A,-0.261009,N/A,-0.070789,N/A,N/A
98,CMA,98.30,N/A,0.464808,N/A,0.336037,N/A,0.110852,N/A,-0.038827,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [48]:
time_periods = [
            'One-Year',
            'Six-Month',
            'Three-Month',
            'One-Month'
                ]
for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row,change_col])
        
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,137.38,N/A,0.106253,50.0,-0.219643,8.0,-0.104639,34.0,-0.001130,59.0,N/A
1,AAL,17.32,N/A,-0.337113,1.0,-0.174364,16.0,-0.003154,65.0,-0.124997,11.0,N/A
2,AAP,215.96,N/A,0.184582,67.0,0.003529,58.0,-0.136586,23.0,-0.067465,30.0,N/A
3,AAPL,161.00,N/A,0.280215,78.0,0.076708,68.0,-0.074109,43.0,-0.077142,25.0,N/A
4,ABBV,165.78,N/A,0.476359,95.0,0.498195,98.0,0.199295,94.0,0.069885,91.0,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
95,CINF,132.47,N/A,0.257220,76.0,0.117668,76.0,0.117335,88.0,0.019044,67.0,N/A
96,CL,77.07,N/A,0.013296,35.0,-0.006220,52.0,-0.101421,35.0,-0.041150,39.0,N/A
97,CLX,135.46,N/A,-0.287987,3.0,-0.206446,10.0,-0.261009,4.0,-0.070789,28.0,N/A
98,CMA,98.30,N/A,0.464808,94.0,0.336037,95.0,0.110852,85.0,-0.038827,42.0,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [49]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,137.38,N/A,0.106253,50.0,-0.219643,8.0,-0.104639,34.0,-0.001130,59.0,37.75
1,AAL,17.32,N/A,-0.337113,1.0,-0.174364,16.0,-0.003154,65.0,-0.124997,11.0,23.25
2,AAP,215.96,N/A,0.184582,67.0,0.003529,58.0,-0.136586,23.0,-0.067465,30.0,44.5
3,AAPL,161.00,N/A,0.280215,78.0,0.076708,68.0,-0.074109,43.0,-0.077142,25.0,53.5
4,ABBV,165.78,N/A,0.476359,95.0,0.498195,98.0,0.199295,94.0,0.069885,91.0,94.5
...,...,...,...,...,...,...,...,...,...,...,...,...
95,CINF,132.47,N/A,0.257220,76.0,0.117668,76.0,0.117335,88.0,0.019044,67.0,76.75
96,CL,77.07,N/A,0.013296,35.0,-0.006220,52.0,-0.101421,35.0,-0.041150,39.0,40.25
97,CLX,135.46,N/A,-0.287987,3.0,-0.206446,10.0,-0.261009,4.0,-0.070789,28.0,11.25
98,CMA,98.30,N/A,0.464808,94.0,0.336037,95.0,0.110852,85.0,-0.038827,42.0,79.0


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [50]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop=True, inplace = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,CF,96.670,N/A,0.840599,100.0,0.848473,100.0,0.385074,98.0,0.188199,100.0,99.5
1,APA,40.120,N/A,0.829228,99.0,0.837382,99.0,0.494650,100.0,0.140412,99.0,99.25
2,BKR,36.280,N/A,0.528971,96.0,0.399208,97.0,0.414696,99.0,0.138180,97.0,97.25
3,ABBV,165.780,N/A,0.476359,95.0,0.498195,98.0,0.199295,94.0,0.069885,91.0,94.5
4,ADM,87.440,N/A,0.444641,93.0,0.312796,94.0,0.238392,96.0,0.061425,87.0,92.5
5,ABC,156.300,N/A,0.353682,88.0,0.225631,89.0,0.226933,95.0,0.055464,86.0,89.5
6,ATO,119.280,N/A,0.250999,74.0,0.282358,93.0,0.148336,92.0,0.086082,94.0,88.25
7,BRK.B,345.850,N/A,0.337114,87.0,0.221074,87.0,0.121729,89.0,0.067560,90.0,88.25
8,CERN,94.474,N/A,0.320059,86.0,0.274107,91.0,0.182003,93.0,0.018484,66.0,84.0
9,ANTM,490.980,N/A,0.399174,91.0,0.274264,92.0,0.055386,73.0,0.029624,71.0,81.75


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [51]:
portfolio_input()

Enter the size of your portfolio:1000000


In [52]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])
hqm_dataframe

C:\Users\conno\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,CF,96.670,206,0.840599,100.0,0.848473,100.0,0.385074,98.0,0.188199,100.0,99.5
1,APA,40.120,498,0.829228,99.0,0.837382,99.0,0.494650,100.0,0.140412,99.0,99.25
2,BKR,36.280,551,0.528971,96.0,0.399208,97.0,0.414696,99.0,0.138180,97.0,97.25
3,ABBV,165.780,120,0.476359,95.0,0.498195,98.0,0.199295,94.0,0.069885,91.0,94.5
4,ADM,87.440,228,0.444641,93.0,0.312796,94.0,0.238392,96.0,0.061425,87.0,92.5
5,ABC,156.300,127,0.353682,88.0,0.225631,89.0,0.226933,95.0,0.055464,86.0,89.5
6,ATO,119.280,167,0.250999,74.0,0.282358,93.0,0.148336,92.0,0.086082,94.0,88.25
7,BRK.B,345.850,57,0.337114,87.0,0.221074,87.0,0.121729,89.0,0.067560,90.0,88.25
8,CERN,94.474,211,0.320059,86.0,0.274107,91.0,0.182003,93.0,0.018484,66.0,84.0
9,ANTM,490.980,40,0.399174,91.0,0.274264,92.0,0.055386,73.0,0.029624,71.0,81.75


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [53]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine = 'xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = "Momentum Strategy", index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [54]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [55]:
column_formats = {
    'A': ['Ticker', string_template], 
    'B': ['Price', dollar_template], 
    'C': ['Number of Shares to Buy', integer_template], 
    'D': ['One-Year Price Return', percent_template], 
    'E': ['One-Year Return Percentile', percent_template], 
    'F': ['Six-Month Price Return', percent_template], 
    'G': ['Six-Month Return Percentile', percent_template], 
    'H': ['Three-Month Price Return', percent_template], 
    'I': ['Three-Month Return Percentile', percent_template], 
    'J': ['One-Month Price Return', percent_template], 
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template]}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
writer.save()

AttributeError: 'list' object has no attribute '_get_xf_index'

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [ ]:
writer.save()